In [1]:
import requests
from requests.exceptions import HTTPError
import re
from bs4 import BeautifulSoup
import os
from time import time
from multiprocessing.pool import ThreadPool

In [2]:
r = requests.get('https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/dados-publicos-cnpj')

In [3]:
def download_html(url, numero_tentativas=2):
    print("Realizando o download da página:", url)
    try:
        req = requests.get(url)
        if req.status_code != 200:
            if numero_tentativas > 0:
                print("Não foi possível realizar o download. Erro:", req.status_code)
                print("\nRealizando nova tentativa:")
                return download_html(url, numero_tentativas - 1)
            else:
                print("Número de tentativas excedidas. Erro: {}".format(req.status_code))
                html = None
                return html
        html = req.text
        return html
    except ConnectionError as e:
        print("Erro no download:", e)
        html = None

In [4]:
html = download_html('https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/dados-publicos-cnpj')

Realizando o download da página: https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/dados-publicos-cnpj


In [5]:
bs = BeautifulSoup(html, "html.parser")

In [6]:
lista_de_links = bs.body.find_all("a", class_="external-link")

In [7]:
lista_de_links

[<a class="external-link" data-tippreview-enabled="true" data-tippreview-image="" data-tippreview-title="" href="http://200.152.38.155/CNPJ/K3241.K03200Y0.D10612.EMPRECSV.zip" target="_self" title="">Dados Abertos CNPJ EMPRESA 01</a>,
 <a class="external-link" data-tippreview-enabled="true" data-tippreview-image="" data-tippreview-title="" href="http://200.152.38.155/CNPJ/K3241.K03200Y1.D10612.EMPRECSV.zip" target="_self" title=""><span>Dados Abertos CNPJ</span> EMPRESA 02</a>,
 <a class="external-link" data-tippreview-enabled="true" data-tippreview-image="" data-tippreview-title="" href="http://200.152.38.155/CNPJ/K3241.K03200Y2.D10612.EMPRECSV.zip" target="_self" title=""><span>Dados Abertos CNPJ</span> EMPRESA 03 </a>,
 <a class="external-link" data-tippreview-enabled="true" data-tippreview-image="" data-tippreview-title="" href="http://200.152.38.155/CNPJ/K3241.K03200Y3.D10612.EMPRECSV.zip" target="_self" title=""><span>Dados Abertos CNPJ</span> EMPRESA 04</a>,
 <a class="external-

In [8]:
for item in lista_de_links:
    print("-->", item.text)

--> Dados Abertos CNPJ EMPRESA 01
--> Dados Abertos CNPJ EMPRESA 02
--> Dados Abertos CNPJ EMPRESA 03 
--> Dados Abertos CNPJ EMPRESA 04
--> Dados Abertos CNPJ EMPRESA 05
--> Dados Abertos CNPJ EMPRESA 06
--> Dados Abertos CNPJ EMPRESA 07 
--> Dados Abertos CNPJ EMPRESA 08
--> Dados Abertos CNPJ EMPRESA 09
--> Dados Abertos CNPJ EMPRESA 10
--> Dados Abertos CNPJ ESTABELECIMENTO 01
--> Dados Abertos CNPJ ESTABELECIMENTO 02
--> Dados Abertos CNPJ ESTABELECIMENTO 03
--> Dados Abertos CNPJ ESTABELECIMENTO 04
--> Dados Abertos CNPJ ESTABELECIMENTO 05
--> Dados Abertos CNPJ ESTABELECIMENTO 06
--> Dados Abertos CNPJ ESTABELECIMENTO 07
--> Dados Abertos CNPJ ESTABELECIMENTO 08
--> Dados Abertos CNPJ ESTABELECIMENTO 09
--> Dados Abertos CNPJ ESTABELECIMENTO 10
--> Dados Abertos CNPJ SÓCIO 01
--> Dados Abertos CNPJ SÓCIO 02
--> Dados Abertos CNPJ SÓCIO 03
--> Dados Abertos CNPJ SÓCIO 04
--> Dados Abertos CNPJ SÓCIO 05
--> Dados Abertos CNPJ SÓCIO 06
--> Dados Abertos CNPJ SÓCIO 07
--> Dados Aber

In [9]:
lista = []
for link in lista_de_links:
    lista.append(link.get("href"))
    print(link.get("href"))

http://200.152.38.155/CNPJ/K3241.K03200Y0.D10612.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y1.D10612.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y2.D10612.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y3.D10612.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y4.D10612.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y5.D10612.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y6.D10612.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y7.D10612.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y8.D10612.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y9.D10612.EMPRECSV.zip
http://200.152.38.155/CNPJ/K3241.K03200Y0.D10612.ESTABELE.zip
http://200.152.38.155/CNPJ/K3241.K03200Y1.D10612.ESTABELE.zip
http://200.152.38.155/CNPJ/K3241.K03200Y2.D10612.ESTABELE.zip
http://200.152.38.155/CNPJ/K3241.K03200Y3.D10612.ESTABELE.zip
http://200.152.38.155/CNPJ/K3241.K03200Y4.D10612.ESTABELE.zip
http://200.152.38.155/CNPJ/K3241.K03200Y5.D10612.ESTABELE.zip
http://2

In [10]:
lista

['http://200.152.38.155/CNPJ/K3241.K03200Y0.D10612.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y1.D10612.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y2.D10612.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y3.D10612.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y4.D10612.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y5.D10612.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y6.D10612.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y7.D10612.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y8.D10612.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y9.D10612.EMPRECSV.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y0.D10612.ESTABELE.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y1.D10612.ESTABELE.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y2.D10612.ESTABELE.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y3.D10612.ESTABELE.zip',
 'http://200.152.38.155/CNPJ/K3241.K03200Y4.D10612.ESTABELE.zip',
 'http://2

In [11]:
def download_url(url, chunk_size=1024):
    path=r'C:\Users\gusta\Documents\IPN\Arquivos CNPJ'
    file = url.split('/')[-1]
    save_path = path + '\\' + file
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [12]:
start = time()

ThreadPool().imap_unordered(download_url, lista)

print(f"Tempo total do download: {time() - start}")

Tempo total do download: 0.015577554702758789
